<a href="https://colab.research.google.com/github/imthegoodboy/LoRa_/blob/main/LoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install transformers datasets peft accelerate bitsandbytes


In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, PeftModel
from datasets import Dataset
import torch


In [12]:
model_name = "gpt2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
# GPT-2 doesn't have a pad token by default, so we add one:
tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)


In [8]:
train_data = [
    {
        "prompt": "User: Hi, my WaveGuard device is overheating.\nSupport:",
        "response": "Hello! I’m sorry you’re facing this issue. Please place the device on a hard surface, ensure the vents are not blocked, and restart it. If it still overheats, I can help you file a warranty request."
    },
    {
        "prompt": "User: My WaveGuard app is stuck on loading.\nSupport:",
        "response": "No worries, I can help with that. First, try closing and reopening the app. If the issue persists, clear the app cache or reinstall it. Let me know what you see after that."
    },
    {
        "prompt": "User: Can I use WaveGuard while traveling?\nSupport:",
        "response": "Yes, you can use WaveGuard while traveling as long as you have an internet connection. For best performance, make sure your connection is stable and your device firmware is updated."
    },
]

def format_example(example):
    # We join prompt + response as one text for causal LM training
    return {
        "text": example["prompt"] + " " + example["response"] + tokenizer.eos_token
    }

formatted_data = [format_example(ex) for ex in train_data]
dataset = Dataset.from_list(formatted_data)


In [19]:
display(dataset)

Dataset({
    features: ['text'],
    num_rows: 3
})

In [20]:
def tokenize_function(example):
    return tokenizer(
        example["text"],
        truncation=True,
        max_length=256,
        padding="max_length",
    )

tokenized_dataset = dataset.map(tokenize_function, batched=False)


Map:   0%|          | 0/3 [00:00<?, ? examples/s]

In [21]:
tokenized_dataset

Dataset({
    features: ['text', 'input_ids', 'attention_mask'],
    num_rows: 3
})

In [22]:
print(tokenized_dataset[0]['input_ids'])
print(tokenizer.decode(tokenized_dataset[0]['input_ids']))

[12982, 25, 15902, 11, 616, 17084, 24502, 3335, 318, 34789, 803, 13, 198, 15514, 25, 18435, 0, 314, 447, 247, 76, 7926, 345, 447, 247, 260, 6476, 428, 2071, 13, 4222, 1295, 262, 3335, 319, 257, 1327, 4417, 11, 4155, 262, 42777, 389, 407, 10226, 11, 290, 15765, 340, 13, 1002, 340, 991, 34789, 1381, 11, 314, 460, 1037, 345, 2393, 257, 18215, 2581, 13, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 5025

In [23]:
lora_config = LoraConfig(
    r=8,                      # rank (how big the LoRA adapter is)
    lora_alpha=16,            # scaling factor
    lora_dropout=0.05,        # dropout on LoRA
    bias="none",
    task_type="CAUSAL_LM",    # type of problem (LLM)
    # optional: which modules in the model get LoRA
    # for GPT2, modules with attention usually contain "c_attn"
    target_modules=["c_attn"]
)


In [24]:
lora_model = get_peft_model(base_model, lora_config)

lora_model.print_trainable_parameters()


trainable params: 294,912 || all params: 124,734,720 || trainable%: 0.2364


/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/layer.py:2174: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [25]:
training_args = TrainingArguments(
    output_dir="./lora-gpt2-waveguard",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=1,
    save_steps=10,
    save_total_limit=2,
)
